In [1]:
# !pip install numpy networkx matplotlib scipy
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy

from session15 import make_all_maps, make_presentation, decompose_endomap, \
    get_loop_size, find_all_cluster_pairs, standardize_presentation, iterate_endomap, \
    make_assignments, join_assignments, follow_endomap, preserves_structure

In [2]:

# in Session 5, the authors discuss a "minimal category" with 2 objects and 8 maps
class BinaryCategory:
    objects = [[0],[0,1]]
    map_count = 0
    maps = []
    # for every possible combination of domain and codomain
    for i,o_0 in enumerate(objects):
        domain_size = len(o_0)
        for j,o_1 in enumerate(objects):
            # add every possible map with that domain and codomain
            codomain_size = len(o_1)
            # maps exist as long as 
            total_maps = codomain_size**domain_size
            print(f"{total_maps} from |{o_0}|={domain_size}  to |{o_1}|={codomain_size}")

            maps_found = make_all_maps(set(o_0),set(o_1))
            assert len(maps_found) == total_maps
            for m in maps_found:
                maps.append({
                    "domain":o_0,
                    "codomain":o_1,
                    "map":m
                })
            map_count += total_maps
   
    print(f"Total maps: {map_count}")
    


1 from |[0]|=1  to |[0]|=1
2 from |[0]|=1  to |[0, 1]|=2
1 from |[0, 1]|=2  to |[0]|=1
4 from |[0, 1]|=2  to |[0, 1]|=2
Total maps: 8


In [3]:
cat2 = BinaryCategory()

In [4]:
cat2.maps

[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
 {'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 1}}]

In [5]:
# I find it interesting that the dict {0:0} shows up twice with
# differing domains and codomains
# effectively we have 3 distinct "points" here:
#  the 0 from [0], 
#  the 0 from [0,1],
#  the 1 from [0,1]

# they add that adding a 3rd object gives 56 maps
class TrinaryCategory:
    objects = [[0],[0,1],[0,1,2]]
    map_count = 0
    maps = []
    # for every possible combination of domain and codomain
    for i,o_0 in enumerate(objects):
        domain_size = len(o_0)
        for j,o_1 in enumerate(objects):
            # add every possible map with that domain and codomain
            codomain_size = len(o_1)
            # maps exist as long as 
            total_maps = codomain_size**domain_size
            print(f"{total_maps} from |{o_0}|={domain_size}  to |{o_1}|={codomain_size}")
            
            maps_found = make_all_maps(set(o_0),set(o_1))
            assert len(maps_found) == total_maps
            for m in maps_found:
                maps.append({
                    "domain":o_0,
                    "codomain":o_1,
                    "map":m
                })
                
            map_count += total_maps

    
    print(f"Total maps: {map_count}")
    

1 from |[0]|=1  to |[0]|=1
2 from |[0]|=1  to |[0, 1]|=2
3 from |[0]|=1  to |[0, 1, 2]|=3
1 from |[0, 1]|=2  to |[0]|=1
4 from |[0, 1]|=2  to |[0, 1]|=2
9 from |[0, 1]|=2  to |[0, 1, 2]|=3
1 from |[0, 1, 2]|=3  to |[0]|=1
8 from |[0, 1, 2]|=3  to |[0, 1]|=2
27 from |[0, 1, 2]|=3  to |[0, 1, 2]|=3
Total maps: 56


In [6]:
cat3 = TrinaryCategory()
cat3.maps

[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 2}},
 {'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1, 2], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1, 2], 'map': {0: 0, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1, 2], 'map': {0: 0, 1: 2}},
 {'domain': [0, 1], 'codomain': [0, 1, 2], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1, 2], 'map': {0: 1, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1, 2], '

In [7]:
# note that now we have a total of 6 maps from [0]:
# the point 0 in [0]
# the point 0 in [0,1]
# the point 1 in [0,1]
# the point 0 in [0,1,2]
# the point 1 in [0,1,2]
# the point 2 in [0,1,2]

# our number of maps goes from 8 to 56
# our number of points goes from 3 to 6

# what happens if we try 4 objects?
class QuaternaryCategory:
    objects = [[0],[0,1],[0,1,2],[0,1,2,3]]
    map_count = 0
    maps = []
    # for every possible combination of domain and codomain
    for i,o_0 in enumerate(objects):
        domain_size = len(o_0)
        for j,o_1 in enumerate(objects):
            # add every possible map with that domain and codomain
            codomain_size = len(o_1)
            # maps exist as long as 
            total_maps = codomain_size**domain_size
            print(f"{total_maps} from |{o_0}|={domain_size}  to |{o_1}|={codomain_size}")
            
            maps_found = make_all_maps(set(o_0),set(o_1))
            assert len(maps_found) == total_maps
            for m in maps_found:
                maps.append({
                    "domain":o_0,
                    "codomain":o_1,
                    "map":m
                })
                
            map_count += total_maps

    
    print(f"Total maps: {map_count}")


1 from |[0]|=1  to |[0]|=1
2 from |[0]|=1  to |[0, 1]|=2
3 from |[0]|=1  to |[0, 1, 2]|=3
4 from |[0]|=1  to |[0, 1, 2, 3]|=4
1 from |[0, 1]|=2  to |[0]|=1
4 from |[0, 1]|=2  to |[0, 1]|=2
9 from |[0, 1]|=2  to |[0, 1, 2]|=3
16 from |[0, 1]|=2  to |[0, 1, 2, 3]|=4
1 from |[0, 1, 2]|=3  to |[0]|=1
8 from |[0, 1, 2]|=3  to |[0, 1]|=2
27 from |[0, 1, 2]|=3  to |[0, 1, 2]|=3
64 from |[0, 1, 2]|=3  to |[0, 1, 2, 3]|=4
1 from |[0, 1, 2, 3]|=4  to |[0]|=1
16 from |[0, 1, 2, 3]|=4  to |[0, 1]|=2
81 from |[0, 1, 2, 3]|=4  to |[0, 1, 2]|=3
256 from |[0, 1, 2, 3]|=4  to |[0, 1, 2, 3]|=4
Total maps: 494


In [8]:
# of these, the ones with domain [0] are:
cat4 = QuaternaryCategory()
points_of_cat4 = list(filter(lambda x: len(x['domain'])==1,cat4.maps))
print(len(points_of_cat4))
points_of_cat4

10


[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 2}},
 {'domain': [0], 'codomain': [0, 1, 2, 3], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1, 2, 3], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2, 3], 'map': {0: 2}},
 {'domain': [0], 'codomain': [0, 1, 2, 3], 'map': {0: 3}}]

In [9]:
# note that our number of "points" follow the sequence of "triangular numbers"
# https://oeis.org/A000217
from session15 import triangular
[triangular(x) for x in range(6)]

[0, 1, 3, 6, 10, 15]

In [10]:
# our number of "maps" follows a different pattern
# so far we have 8, 56, 494
# this is OEIS sequence A086787
# https://oeis.org/A086787

def map_count_sequence(n):
    return sum([
       sum([(i+1)**(j+1) for j in range(n)]) for i in range(n)
    ])

list(map(map_count_sequence,range(6)))


[0, 1, 8, 56, 494, 5699]

In [11]:
# Let's break down the "philosophical role of N"
# start by collecting the points of our minimal category
points_of_cat2 = list(filter(lambda x: len(x['domain'])==1,cat2.maps))
print(len(points_of_cat2))
points_of_cat2

3


[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}}]

In [12]:
# maps from 2 -> X represent pairs of points
pairs_of_points_of_cat2 = list(filter(lambda x: len(x['domain'])==2,cat2.maps))
print(len(pairs_of_points_of_cat2))
pairs_of_points_of_cat2

5


[{'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 1}}]

In [13]:
# only 5? interesting... I expected with 3 points we should have 3 nPr 2 possible pairs
# let's make a more complete list
def cartesian_product_of_lists(a,b):
    return [(a[i],b[j+i]) for i in range(len(a)) for j in range(len(b)-i)]
all_possible_point_pairs =  cartesian_product_of_lists(points_of_cat2,points_of_cat2)
print(len(all_possible_point_pairs))
all_possible_point_pairs

6


[({'domain': [0], 'codomain': [0], 'map': {0: 0}},
  {'domain': [0], 'codomain': [0], 'map': {0: 0}}),
 ({'domain': [0], 'codomain': [0], 'map': {0: 0}},
  {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}}),
 ({'domain': [0], 'codomain': [0], 'map': {0: 0}},
  {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}}),
 ({'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
  {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}}),
 ({'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
  {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}}),
 ({'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
  {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}})]

In [14]:

# now let's look for maps X -> 2
properties_of_points_of_cat2 = list(filter(lambda x: len(x['codomain'])==2,cat2.maps))
print(len(properties_of_points_of_cat2))
properties_of_points_of_cat2

6


[{'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 1}}]

In [15]:
# now lets look for endomaps 2 --> 2
endomaps_on_2 = list(filter(
    lambda x: len(x['domain'])==2 and len(x['codomain'])==2,cat2.maps
))
print(len(endomaps_on_2))
endomaps_on_2

4


[{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 1}}]

In [16]:
# which of our endomaps on 2 have a fixed point?

def has_fixed_point(endo):
    for k in endo['domain']:
        if endo['map'][k] == k:
            return True
    return False

have_a_fixed_point = []
for x in endomaps_on_2:
    have_a_fixed_point.append(has_fixed_point(x))

have_a_fixed_point


[True, True, False, True]

In [17]:
# okay, some of our maps can be expressed as compositions
# let's try to find out how
def are_same_map(a,b):
    if a['domain'] != b['domain']:
        return False
    if a['codomain'] != b['codomain']:
        return False
    for k,v in zip(a['map'].keys(),a['map'].values()):
        if b['map'][k] != v:
            return False
    return True

def compose_maps(a,b):
    assert a['codomain'] == b['domain'], "domain/codomain mismatch"
    return {
        'domain':a['domain'],
        'codomain':b['codomain'],
        'map':{
            k:b['map'][v] for k,v in zip(a['map'].keys(),a['map'].values())
        }
    }

def search_for_compositions(miniCat):
    compositions = []
    for i,m1 in enumerate(miniCat.maps):
        for j,m2 in enumerate(miniCat.maps):
            # look for valid compositions
            if m1['codomain'] != m2['domain']:
                continue
            composition = compose_maps(m1,m2)
            # check the composition against other maps
            comp_found = -1
            for k,m3 in enumerate(miniCat.maps):
                if are_same_map(composition,m3):
                    comp_found = k
                    break
            compositions.append((i,j,k))
    return compositions
     
comps2 = search_for_compositions(cat2)
comps2

[(0, 0, 0),
 (0, 1, 1),
 (0, 2, 2),
 (1, 3, 0),
 (1, 4, 1),
 (1, 5, 1),
 (1, 6, 2),
 (1, 7, 2),
 (2, 3, 0),
 (2, 4, 1),
 (2, 5, 2),
 (2, 6, 1),
 (2, 7, 2),
 (3, 0, 3),
 (3, 1, 4),
 (3, 2, 7),
 (4, 3, 3),
 (4, 4, 4),
 (4, 5, 4),
 (4, 6, 7),
 (4, 7, 7),
 (5, 3, 3),
 (5, 4, 4),
 (5, 5, 5),
 (5, 6, 6),
 (5, 7, 7),
 (6, 3, 3),
 (6, 4, 4),
 (6, 5, 6),
 (6, 6, 5),
 (6, 7, 7),
 (7, 3, 3),
 (7, 4, 4),
 (7, 5, 7),
 (7, 6, 4),
 (7, 7, 7)]

In [18]:
# some maps are idempotent & compose to get self
def find_idempotents(cat):
    idempotents = []
    comps = search_for_compositions(cat)
    for i,c in enumerate(comps):
        if c[0] == c[1] == c[2]:
            print(cat.maps[c[0]])
            idempotents.append(c[0])
    return idempotents
idempotents2 = find_idempotents(cat2)
idempotents2

{'domain': [0], 'codomain': [0], 'map': {0: 0}}
{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}}
{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}}
{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 1}}


[0, 4, 5, 7]

In [19]:
# some maps are "identity" maps to start with
def find_identity_maps(cat):
    identities = []
    for i,m in enumerate(cat.maps):
        if m['domain'] != m['codomain']:
            continue
        is_identity = True
        for k,v in zip(m['map'].keys(),m['map'].values()):
            if k != v:
                is_identity = False
                break
        if is_identity:
            print(f"Map {i} is an identity: {m}")
            identities.append(i)
    return identities
identities2 = find_identity_maps(cat2)
identities2

Map 0 is an identity: {'domain': [0], 'codomain': [0], 'map': {0: 0}}
Map 5 is an identity: {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}}


[0, 5]

In [20]:
# we can also group by the composition
maps_by_comp = {}
for c in comps2:
    if c[2] not in maps_by_comp:
        maps_by_comp[c[2]] = []
    maps_by_comp[c[2]].append((c[0],c[1]))
maps_by_comp
        

{0: [(0, 0), (1, 3), (2, 3)],
 1: [(0, 1), (1, 4), (1, 5), (2, 4), (2, 6)],
 2: [(0, 2), (1, 6), (1, 7), (2, 5), (2, 7)],
 3: [(3, 0), (4, 3), (5, 3), (6, 3), (7, 3)],
 4: [(3, 1), (4, 4), (4, 5), (5, 4), (6, 4), (7, 4), (7, 6)],
 7: [(3, 2), (4, 6), (4, 7), (5, 7), (6, 7), (7, 5), (7, 7)],
 5: [(5, 5), (6, 6)],
 6: [(5, 6), (6, 5)]}

In [21]:
# it looks like the set of maps 0,1,2,3,6
# might be sufficient to generate the rest
min_maps2 = [cat2.maps[x] for x in [0,1,2,3,6]]
min_maps2

[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
 {'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}}]

In [22]:
# lets verify that we can make all 8 maps from these
comps2 = []
for i,m1 in enumerate(min_maps2):
    for j,m2 in enumerate(cat2.maps):
        # look for valid compositions
        if m1['codomain'] != m2['domain']:
            continue
        composition = compose_maps(m1,m2)
        # check the composition against other maps
        comp_found = -1
        for k,m3 in enumerate(cat2.maps):
            if are_same_map(composition,m3):
                comp_found = k
                break
        comps2.append((i,j,k))
produced2 = list(set([x[2] for x in comps2]))
len(produced2)


8

In [23]:
# so our first 3 key maps are the "points" 1->X
print(points_of_cat2 == min_maps2[0:3])
points_of_cat2

True


[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}}]

In [24]:
# our 4th key map is the unique "terminator" 2->1
min_maps2[3]


{'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}}

In [25]:
# and our 5th is the antipodal map 2->2
min_maps2[4]

{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}}

In [26]:
# lets check cat3 to see if there's a pattern
points_of_cat3 = list(filter(lambda x: len(x['domain'])==1,cat3.maps))
print(f"Cat3 has {len(points_of_cat3)} points: \n{points_of_cat3}")
comps3 = search_for_compositions(cat3)
print(f"Cat3 has {len(comps3)} compositions")
terminators3 = list(filter(lambda x: len(x['codomain'])==1,cat3.maps))
print(f"Cat3 has {len(terminators3)} terminator maps: \n{terminators3}")
identities3 = find_identity_maps(cat3)
print(f"Cat3 has {len(identities3)} identity maps")
idempotents3 = find_idempotents(cat3)
print(f"Cat3 has {len(idempotents3)} idempotents")

Cat3 has 6 points: 
[{'domain': [0], 'codomain': [0], 'map': {0: 0}}, {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}}, {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}}, {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 0}}, {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 1}}, {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 2}}]
Cat3 has 1618 compositions
Cat3 has 3 terminator maps: 
[{'domain': [0], 'codomain': [0], 'map': {0: 0}}, {'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}}, {'domain': [0, 1, 2], 'codomain': [0], 'map': {0: 0, 1: 0, 2: 0}}]
Map 0 is an identity: {'domain': [0], 'codomain': [0], 'map': {0: 0}}
Map 8 is an identity: {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 1}}
Map 48 is an identity: {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 2, 0: 0, 1: 1}}
Cat3 has 3 identity maps
{'domain': [0], 'codomain': [0], 'map': {0: 0}}
{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 0, 1: 0}}
{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 

In [27]:
# lets see if we can find all "endomaps with no fixed point"
def find_endomaps_without_fixed_point(cat):
    # for each possible domain
    output = []
    for o in cat.objects:
        # find all endomap on that domain
        endomaps = list(filter(lambda x: x['domain']==o and x['codomain']==o,cat.maps))
        # filter ones with no fixed points
        even_endomaps = list(filter(lambda x: not has_fixed_point(x),endomaps))
        output.extend(even_endomaps)
    return output

non_fixed3 = find_endomaps_without_fixed_point(cat3)
non_fixed3

[{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 1, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 1, 1: 2}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 2, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 2, 1: 2}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 1, 0: 1, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 1, 0: 1, 1: 2}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 1, 0: 2, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 1, 0: 2, 1: 2}}]

In [28]:
# lets also see if we can narrow these down by the size of the output space
def is_one_to_one(m):
    return len(m['domain']) == len(set(m['map'].values()))

non_fixed_one_to_one3 = list(filter(is_one_to_one,non_fixed3))
non_fixed_one_to_one3

[{'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 1, 1: 2}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 1, 0: 2, 1: 0}}]

In [29]:
# note that the last two maps compose to form an identity
compose_maps(non_fixed_one_to_one3[1],non_fixed_one_to_one3[2])

{'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 2, 0: 0, 1: 1}}

In [30]:
# and that one produces the other through self-composition:
compose_maps(non_fixed_one_to_one3[1],non_fixed_one_to_one3[1])

{'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 1, 0: 2, 1: 0}}

In [34]:
# so, putting this all together, I think we can build all cat3 maps from:
#    the "points", the "terminators", and the "loops"
#  and the compositions thereof
min_maps3 = points_of_cat3
min_maps3.append(non_fixed_one_to_one3[0])
min_maps3.append(non_fixed_one_to_one3[1])
for m1 in terminators3:
    map_found = False
    for m2 in min_maps3:
        if are_same_map(m1,m2):
            map_found = True
    if not map_found:
        min_maps3.append(m1)
print(len(min_maps3))
min_maps3

12


[{'domain': [0], 'codomain': [0], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 0}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 1}},
 {'domain': [0], 'codomain': [0, 1, 2], 'map': {0: 2}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 1, 1: 2}},
 {'domain': [0, 1], 'codomain': [0], 'map': {0: 0, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0], 'map': {0: 0, 1: 0, 2: 0}},
 {'domain': [0, 1], 'codomain': [0, 1], 'map': {0: 1, 1: 0}},
 {'domain': [0, 1, 2], 'codomain': [0, 1, 2], 'map': {2: 0, 0: 1, 1: 2}}]

In [32]:
# now lets check if we made all of them
def produced_by_compositions(cat,min_maps):
    comps = []
    for i,m1 in enumerate(min_maps):
        for j,m2 in enumerate(cat.maps):
            # look for valid compositions
            if m1['codomain'] != m2['domain']:
                continue
            composition = compose_maps(m1,m2)
            # check the composition against other maps
            comp_found = -1
            for k,m3 in enumerate(cat.maps):
                if are_same_map(composition,m3):
                    comp_found = k
                    break
            comps.append((i,j,k))

    return  list(set([x[2] for x in comps]))
prod3 = produced_by_compositions(cat3,min_maps3)
len(prod3)

56

In [33]:
# i think that's all of them!
len(set(prod3)) == len(cat3.maps)

True